Jupyter Notebook for Selecting Lipid MRMs and OzESIs after a Neutral Loss Scan

In [2]:
#Import all the necessary libraries
import pymzml
import csv
import os
import pandas as pd
import numpy as np
import math
from matplotlib import pyplot as plt
import re
import plotly.express as px
from IPython.display import Image

No module named 'ms_deisotope._c.averagine' averagine
No module named 'ms_deisotope._c.scoring'
No module named 'ms_deisotope._c.deconvoluter_base'
No module named 'ms_deisotope._c.deconvoluter_base'
No module named 'ms_deisotope._c.deconvoluter_base'


First perform a neutral loss scan and obtain pecursor ion m/z values and the neutral loss. Upload an excel spreadsheet with these values listed in two columns.

![title](Figures/search_list.png)

A database of known lipid MRMs will be parsed and matched. And the rule-based method for OzESI will be used. 


![title](Figures/OzESI_results.png)

Load MRM databases 1: SUPPLE_2.XLS and 2:ListMRMs.csv, and then save as a pandas dataframe. 

In [8]:
# #loop through all sheets in SUPPLE_2.XLS and make a df of Compound Name, Parent Ion, and Product Ion
mrm_list_new = pd.read_excel('lipid_database/Lipid_Database.xlsx', sheet_name = None)
mrm_list_new = pd.concat(mrm_list_new, ignore_index=True)
mrm_list_offical = mrm_list_new[['Compound Name', 'Parent Ion', 'Product Ion']]
#mrm_list_official = mrm_list_new.loc[:, ['Compound Name', 'Parent Ion', 'Product Ion']]
#Add underscore to middle of columns names
mrm_list_offical.columns = mrm_list_offical.columns.str.replace(' ', '_')
#round the Parent Ion and Product Ion to 1 decimal place
mrm_list_offical.loc[:, 'Parent_Ion'] = mrm_list_offical['Parent_Ion'].round(0)
mrm_list_offical.loc[:, 'Product_Ion'] = mrm_list_offical['Product_Ion'].round(0)
# Create transition column by combining Parent Ion and Product Ion with arrow between numbers
mrm_list_offical.loc[:, 'Transition'] = mrm_list_offical['Parent_Ion'].astype(str) + ' -> ' + mrm_list_offical['Product_Ion'].astype(str)
# Change column compound name to lipid
mrm_list_offical= mrm_list_offical.rename(columns={'Compound_Name': 'Lipid'})
mrm_database = mrm_list_offical



# # print(mrm_list_offical.head(25))
# # print(df_mrm.head(25))

# def read_mrm_list(filename):
#     mrm_list_new = pd.read_excel(filename, sheet_name=None)
#     mrm_list_new = pd.concat(mrm_list_new, ignore_index=True)
#     mrm_list_offical = mrm_list_new[['Compound Name', 'Parent Ion', 'Product Ion', 'Class']]
#     # Add underscore to middle of columns names
#     mrm_list_offical.columns = mrm_list_offical.columns.str.replace(' ', '_')
#     # Round Parent Ion and Product Ion to 1 decimal place
#     mrm_list_offical['Parent_Ion'] = np.floor(mrm_list_offical['Parent_Ion'].round(0))
#     mrm_list_offical['Product_Ion'] = np.floor(mrm_list_offical['Product_Ion'].round(0))
#     # Create transition column by combining Parent Ion and Product Ion with arrow between numbers
#     mrm_list_offical['Transition'] = mrm_list_offical['Parent_Ion'].astype(str) + ' -> ' + mrm_list_offical['Product_Ion'].astype(str)
#     # Change column compound name to lipid
#     mrm_list_offical = mrm_list_offical.rename(columns={'Compound_Name': 'Lipid'})
#     # Make a column called Class match lipid column to lipid types
#     return mrm_list_offical

# mrm_database = read_mrm_list('lipid_database/Lipid_Database.xlsx')
mrm_database.tail()

/tmp/ipykernel_848/1191973850.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mrm_list_offical.loc[:, 'Parent_Ion'] = mrm_list_offical['Parent_Ion'].round(0)
/tmp/ipykernel_848/1191973850.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mrm_list_offical.loc[:, 'Product_Ion'] = mrm_list_offical['Product_Ion'].round(0)
/tmp/ipykernel_848/1191973850.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


,Lipid,Parent_Ion,Product_Ion,Transition
3264,STD_15:0-18:1(d7) PI (NH4 Salt),848.0,570.0,848.0 -> 570.0
3265,STD_15:0-18:1(d7) PS (Na Salt),756.0,570.0,756.0 -> 570.0
3266,STD_15:0-18:1(d7)-15:0 TAG,830.0,570.0,830.0 -> 570.0
3267,STD_18:1(d7) Chol Ester,676.0,369.0,676.0 -> 369.0
3268,STD_d18:1-18:1(d9) SM,739.0,184.0,739.0 -> 184.0


Input search list excel sheet as discussed in the introduction. Save the data into a dataframe and then search the databases for matches and output them into a found_list dataframe. 

In [9]:
search_list = pd.read_excel('./data_OzESI/search/Practice_LiverLD_MUFAs_2.xlsx')
found_list = pd.DataFrame(columns=['Lipid','Parent_Ion','Product_Ion','Transition'])
print(search_list.head(10))
search_list['Product_Ion'] = search_list['Parent_Ion'] - search_list['neutral_loss']
#Round search list columns to 0 deciaml places
search_list['Parent_Ion'] = search_list['Parent_Ion'].round(0)
search_list['Product_Ion'] = search_list['Product_Ion'].round(0)

print(search_list.head())

#Search for matches between mrm_list_offical and search_list for Parent Ion and Product Ion
for i in range(len(mrm_list_offical)):
    for j in range(len(search_list)):
        if mrm_list_offical.loc[i,'Parent_Ion'] == search_list.loc[j,'Parent_Ion'] and mrm_list_offical.loc[i,'Product_Ion'] == search_list.loc[j,'Product_Ion']:
            found_list = found_list.append(mrm_list_offical.loc[i,:])



# found_list.to_excel('./data_OzESI/data_csv/found_list.xlsx', index=False)      
found_list.head(25)

   Parent_Ion  neutral_loss
0       846.8         299.2
1       848.8         299.2
2       850.9         299.2
3       870.8         299.2
4       872.8         299.2
5       874.8         299.2
6       876.8         299.2
7       878.9         299.2
8       896.9         299.2
9       898.9         299.2
   Parent_Ion  neutral_loss  Product_Ion
0       847.0         299.2        548.0
1       849.0         299.2        550.0
2       851.0         299.2        552.0
3       871.0         299.2        572.0
4       873.0         299.2        574.0


/tmp/ipykernel_848/897860761.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  found_list = found_list.append(mrm_list_offical.loc[i,:])
/tmp/ipykernel_848/897860761.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  found_list = found_list.append(mrm_list_offical.loc[i,:])
/tmp/ipykernel_848/897860761.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  found_list = found_list.append(mrm_list_offical.loc[i,:])
/tmp/ipykernel_848/897860761.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  found_list = found_list.append(mrm_list_offical.loc[i,:])
/tmp/ipykernel_848/897860761.py:15: FutureWarning: The frame.append method is deprec

,Lipid,Parent_Ion,Product_Ion,Transition
1360,"[TG(53:8),TG(52:1)]_FA18:1",879.0,580.0,879.0 -> 580.0
1361,"[TG(53:9),TG(52:2)]_FA18:1",877.0,578.0,877.0 -> 578.0
1369,"[TG(57:10),TG(56:3)]_FA18:1",931.0,632.0,931.0 -> 632.0
1444,"[TG(55:10),TG(54:3)]_FA18:1",903.0,604.0,903.0 -> 604.0
1445,"[TG(55:11),TG(54:4)]_FA18:1",901.0,602.0,901.0 -> 602.0
1447,"[TG(55:9),TG(54:2)]_FA18:1",905.0,606.0,905.0 -> 606.0


Create a dataframe of DB_position and aldehyde ion based off lipid rules. Save in dataframe df_OzESI

In [10]:
df_OzESI = pd.DataFrame(columns=['DB_Position','Aldehyde_Ion'])

for i in range(3,21):
    df_OzESI.loc[i,'DB_Position'] = i
    df_OzESI.loc[i,'Aldehyde_Ion'] = 26 + (14 * (i-3))

df_OzESI.head(25)


,DB_Position,Aldehyde_Ion
3,3,26
4,4,40
5,5,54
6,6,68
7,7,82
8,8,96
9,9,110
10,10,124
11,11,138
12,12,152


Input the requested OzESI n# in the OzESI_list. Based off the previously found data in the found_list, the script will provide the correct m/z value with each n# for each lipid in the found list dataframe

In [11]:
OzESI_list = [7,9,11]
#create columns in found list called n-# based on the numbers in OzESI_list
for i in OzESI_list:
    found_list['n-' + str(i)] = ''
print(found_list.head(25))

# Parent_Ion - Aldehyde_Ion *

# Subtract the aldehyde ion from the parent ion with the corresponding DB position and add the result to the found_list dataframe
for i in range(3,len(df_OzESI)):
    for j in range(len(found_list)):
        #if df_OzESI.loc[i,'DB_Position']:
        for k in range(4,7):
            if df_OzESI.loc[i,'DB_Position'] == OzESI_list[k-4]:
                found_list.iloc[j,k] = found_list.iloc[j,1] - df_OzESI.loc[i,'Aldehyde_Ion'] + 1

        
found_list.head(25)

                            Lipid  Parent_Ion  Product_Ion      Transition  \
1360   [TG(53:8),TG(52:1)]_FA18:1       879.0        580.0  879.0 -> 580.0   
1361   [TG(53:9),TG(52:2)]_FA18:1       877.0        578.0  877.0 -> 578.0   
1369  [TG(57:10),TG(56:3)]_FA18:1       931.0        632.0  931.0 -> 632.0   
1444  [TG(55:10),TG(54:3)]_FA18:1       903.0        604.0  903.0 -> 604.0   
1445  [TG(55:11),TG(54:4)]_FA18:1       901.0        602.0  901.0 -> 602.0   
1447   [TG(55:9),TG(54:2)]_FA18:1       905.0        606.0  905.0 -> 606.0   

     n-7 n-9 n-11  
1360               
1361               
1369               
1444               
1445               
1447               


,Lipid,Parent_Ion,Product_Ion,Transition,n-7,n-9,n-11
1360,"[TG(53:8),TG(52:1)]_FA18:1",879.0,580.0,879.0 -> 580.0,798.0,770.0,742.0
1361,"[TG(53:9),TG(52:2)]_FA18:1",877.0,578.0,877.0 -> 578.0,796.0,768.0,740.0
1369,"[TG(57:10),TG(56:3)]_FA18:1",931.0,632.0,931.0 -> 632.0,850.0,822.0,794.0
1444,"[TG(55:10),TG(54:3)]_FA18:1",903.0,604.0,903.0 -> 604.0,822.0,794.0,766.0
1445,"[TG(55:11),TG(54:4)]_FA18:1",901.0,602.0,901.0 -> 602.0,820.0,792.0,764.0
1447,"[TG(55:9),TG(54:2)]_FA18:1",905.0,606.0,905.0 -> 606.0,824.0,796.0,768.0


In [18]:
#print datatype of Parent_Ion column and n-7 column
print(found_list['Parent_Ion'].dtypes)
print(found_list['n-7'].dtypes)

#change datatype of all n-# columns to float64
for i in range(4,7):
    found_list.iloc[:,i] = found_list.iloc[:,i].astype('float64')

print(found_list['Parent_Ion'].dtypes)
print(found_list['n-7'].dtypes)
print(found_list['Product_Ion'].dtypes)



float64
object
float64
float64
float64


/tmp/ipykernel_848/799680419.py:7: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  found_list.iloc[:,i] = found_list.iloc[:,i].astype('float64')


In [19]:
print(found_list['n-7'].dtypes)
print(found_list['Product_Ion'].dtypes)

float64
float64


Save the dataframe as an excel sheet that can be downloaded and given back to the mass spec chemist

In [8]:
from datetime import date
i = 0
while i < 100:
    if not os.path.exists('./data_OzESI/data_excel/OzESI_Selections_{}_.xlsx'.format(date.today())):
        found_list.to_excel('./data_OzESI/data_excel/OzESI_Selections_{}_.xlsx'.format(date.today()), index=False)
        break
    elif not os.path.exists('./data_OzESI/data_excel/OzESI_Selections_{}_'.format(date.today()) + str(i) + '.xlsx'):
        found_list.to_excel('./data_OzESI/data_excel/OzESI_Selections_{}_'.format(date.today()) + str(i) + '.xlsx', index=False)
        break
    else:
        i += 1


In [65]:
matching_list = pd.DataFrame(columns=['Lipid','Parent_Ion','Product_Ion','Class','Transition'])

#if n-7 and product ion are the same for different lipids, add them to the matching list
for i in range(len(found_list)):
    for j in range(len(found_list)):
        if found_list.iloc[i,5] == found_list.iloc[j,2] and i != j:
            matching_list = matching_list.append(found_list.iloc[i,:])

matching_list.head(25)

,Lipid,Parent_Ion,Product_Ion,Class,Transition


In [20]:
# Group the DataFrame by 'Product_Ion' and 'n-7'
grouped = found_list.groupby(['Product_Ion', 'n-7'])
print(grouped.tail())

# Create an empty dictionary to store the matching lipids
matching_lipids = {}

# Iterate over the groups
for group_name, group_df in grouped:
    # Get the list of lipids in the group
    lipids = list(group_df['Lipid'])
    
    # Check if there are multiple lipids in the group
    if len(lipids) > 1:
        # Add the lipids to the matching_lipids dictionary
        matching_lipids[group_name] = lipids

# Print the matching lipids
print(matching_lipids)


                            Lipid  Parent_Ion  Product_Ion      Transition  \
1360   [TG(53:8),TG(52:1)]_FA18:1       879.0        580.0  879.0 -> 580.0   
1361   [TG(53:9),TG(52:2)]_FA18:1       877.0        578.0  877.0 -> 578.0   
1369  [TG(57:10),TG(56:3)]_FA18:1       931.0        632.0  931.0 -> 632.0   
1444  [TG(55:10),TG(54:3)]_FA18:1       903.0        604.0  903.0 -> 604.0   
1445  [TG(55:11),TG(54:4)]_FA18:1       901.0        602.0  901.0 -> 602.0   
1447   [TG(55:9),TG(54:2)]_FA18:1       905.0        606.0  905.0 -> 606.0   

        n-7    n-9   n-11  
1360  798.0  770.0  742.0  
1361  796.0  768.0  740.0  
1369  850.0  822.0  794.0  
1444  822.0  794.0  766.0  
1445  820.0  792.0  764.0  
1447  824.0  796.0  768.0  
{}


In [14]:
# Group the DataFrame by 'Product_Ion' and 'n-7'
grouped = found_list.groupby(['Product_Ion', 'n-11'])
# print(grouped.tail())
# Create an empty dictionary to store the matching lipids
matching_lipids = {}

# Iterate over the groups
for group_name, group_df in grouped:
    # Get the list of lipids in the group
    lipids = list(group_df['Lipid'])
    
    # Check if there are multiple lipids in the group
    if len(lipids) > 1:
        # Add the lipids to the matching_lipids dictionary
        matching_lipids[group_name] = lipids

# Print the matching lipids
print(matching_lipids)

{}


In [73]:
# Group the DataFrame by 'Product_Ion' and 'n-7'
grouped = found_list.groupby(['Product_Ion', 'n-7'])

# Create an empty DataFrame to store the matching lipids
matching_lipids_df = pd.DataFrame(columns=['Product_Ion', 'n-7', 'Matching_Lipids'])

# Iterate over the groups
for group_name, group_df in grouped:
    # Get the list of lipids in the group
    lipids = list(group_df['Lipid'])
    
    # Check if there are multiple lipids in the group
    if len(lipids) > 1:
        # Add the group to the matching_lipids_df DataFrame
        matching_lipids_df = matching_lipids_df.append({
            'Product_Ion': group_name[0],
            'n-7': group_name[1],
            'Matching_Lipids': lipids
        }, ignore_index=True)

# Print the matching lipids DataFrame
print(matching_lipids_df.head(25))
matching_lipids_df.to_excel('./data_OzESI/data_excel/FA 18:1.xlsx', index=False)


Empty DataFrame
Columns: [Product_Ion, n-7, Matching_Lipids]
Index: []


In [24]:
# Group the DataFrame by 'Product_Ion' and 'n-7'
grouped = found_list.groupby(['Product_Ion', 'n-11'])

# Create an empty DataFrame to store the matching lipids
matching_lipids_df = pd.DataFrame(columns=['Product_Ion', 'n-11', 'Matching_Lipids'])

# Iterate over the groups
for group_name, group_df in grouped:
    # Get the list of lipids in the group
    lipids = list(group_df['Lipid'])
    
    # Check if there are multiple lipids in the group
    if len(lipids) > 1:
        # Add the group to the matching_lipids_df DataFrame
        matching_lipids_df = matching_lipids_df.append({
            'Product_Ion': group_name[0],
            'n-11': group_name[1],
            'Matching_Lipids': lipids
        }, ignore_index=True)

# Print the matching lipids DataFrame
matching_lipids_df.head(25)
matching_lipids_df.to_excel('./data_OzESI/data_excel/FA 20:1.xlsx', index=False)


/tmp/ipykernel_31304/4196542277.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matching_lipids_df = matching_lipids_df.append({
/tmp/ipykernel_31304/4196542277.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matching_lipids_df = matching_lipids_df.append({
/tmp/ipykernel_31304/4196542277.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matching_lipids_df = matching_lipids_df.append({
/tmp/ipykernel_31304/4196542277.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matching_lipids_df = matching_lipids_df.append({
/tmp/ipykernel_31304/4196542277.py:15: FutureWarning: The frame.append method is deprecated and will be remo